# Creación de las funciones

Creamos las funciones y las probamos

In [1]:
import pandas as pd
import numpy as np

Importamos los datasets que generamos en 'ETL y EDA/Functions_Prep.ipynb'

In [2]:
playtimegenre = pd.read_csv('PlayTimeGenre.csv')
userforgenre = pd.read_csv('UserForGenre.csv')
usersrecommend = pd.read_csv('UsersRecommend.csv')
usersworstdeveloper = pd.read_csv('UsersWorstDeveloper.csv')
sentimentanalysis = pd.read_csv('sentiment_analysis.csv')
recomendacionjuego = pd.read_csv('product.csv')

PlayTimeGenre

In [25]:
def PlayTimeGenre(genero: str):
    filtered_df = playtimegenre[playtimegenre['genres'] == genero]
    if filtered_df.empty:
        error_message = {"Error": "Género incorrecto"}
        return error_message
    grouped_df = filtered_df.groupby('release_year')['playtime_forever'].sum()
    max_year = int(grouped_df.idxmax())
    result = {"Año de lanzamiento con más horas jugadas para Género {}".format(genero): max_year}
    return result

In [107]:
PlayTimeGenre('Adventure')

{'Año de lanzamiento con más horas jugadas para Género Adventure': 2011}

UserForGenre

In [32]:
def UserForGenre(genero: str):
    genre_df = userforgenre[userforgenre['genres'] == genero]
    if genre_df.empty:
        error_message = {"Error": "Género no encontrado"}
        return error_message
    user_hours = genre_df.groupby('user_id')['playtime_forever'].sum()
    max_user = user_hours.idxmax()
    max_user_df = genre_df[genre_df['user_id'] == max_user]
    hours_by_year = max_user_df.groupby('release_year')['playtime_forever'].sum().reset_index()
    hours_list = [{'Año': year, 'Horas': hours} for year, hours in zip(hours_by_year['release_year'], hours_by_year['playtime_forever'])]
    result = {
        "Usuario con más horas jugadas para Género {}".format(genero): max_user,
        "Horas jugadas": hours_list
    }
    return result

In [35]:
UserForGenre('Design & Illustration')

{'Usuario con más horas jugadas para Género Design & Illustration': 'ScottyG555',
 'Horas jugadas': [{'Año': 2015, 'Horas': 2805}]}

UsersRecommend

In [69]:
def UsersRecommend(año: int):
    if not isinstance(año, int):
        error_message = {"Error": "Debe ingresar un año en formato entero"}
        return error_message
    filtered_df = usersrecommend[(usersrecommend['year'] == año) & (usersrecommend['recommend_score'] > 0)]
    if filtered_df.empty:
        error_message = {"Error": "No hay recomendaciones para el año especificado"}
        return error_message
    sorted_df = filtered_df.sort_values(by='recommend_score', ascending=False)
    top_3 = sorted_df.head(3)
    result = [{"Puesto 1": top_3.iloc[0]['app_name']},
              {"Puesto 2": top_3.iloc[1]['app_name']},
              {"Puesto 3": top_3.iloc[2]['app_name']}]

    return result

In [73]:
UsersRecommend(2010)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Killing Floor'},
 {'Puesto 3': 'Alien Swarm'}]

UsersWorstDeveloper

In [80]:
def UsersWorstDeveloper(año: int):
    if not isinstance(año, int):
        error_message = {"Error": "Debe ingresar un año en formato entero"}
        return error_message
    filtered_df = usersworstdeveloper[(usersworstdeveloper['year'] == año) & (usersworstdeveloper['bad_reviews'] > 0)]
    if filtered_df.empty:
        error_message = {"Error": "No hay reseñas negativas para el año especificado"}
        return error_message
    developer_stats = filtered_df.groupby('developer').agg({'bad_reviews': 'sum', 'recommend_score': ['min', 'sum']})
    sorted_developers = developer_stats.sort_values(by=[('bad_reviews', 'sum'), ('recommend_score', 'min'), ('recommend_score', 'sum')],
                                                    ascending=[False, True, True])
    top_developers = sorted_developers.head(3)
    result = [{"Puesto {}".format(i + 1): developer}
              for i, developer in enumerate(top_developers.index)]
    return result

In [83]:
UsersWorstDeveloper(2015)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Bohemia Interactive'},
 {'Puesto 3': 'Facepunch Studios'}]

Sentiment Analysis

In [103]:
def sentiment_analysis(desarrolladora: str):
    # Filtrar el DataFrame por la empresa desarrolladora
    empresa_df = sentimentanalysis[sentimentanalysis['developer'] == desarrolladora]
    if empresa_df.empty:
        error_message = {'Error': 'Desarrolladora incorrecta'}
        return error_message

    # Calcular la cantidad total de cada categoría de sentimiento y convertimos a entero
    total_negativas = int(empresa_df['bad_reviews'].sum())
    total_neutrales = int(empresa_df['neutral_reviews'].sum())
    total_positivas = int(empresa_df['good_reviews'].sum())

    # Crear el diccionario de retorno
    resultado = {
        desarrolladora: {
            'Negative': total_negativas,
            'Neutral': total_neutrales,
            'Positive': total_positivas
        }
    }

    return resultado

In [106]:
sentiment_analysis('Valve')

{'Valve': {'Negative': 1597, 'Neutral': 2558, 'Positive': 5417}}

In [3]:
def recomendacion_juego(game_id: int):
    if not isinstance(game_id, int):
        error_message = {"Error": "El Id del juego debe ser un valor entero"}
        return error_message
    if game_id not in recomendacionjuego['id'].values:
        error_message = {"Error": "Id de juego incorrecto"}
        return error_message
    path = 'cosine_sim.npy'
    cosine_sim = np.load(path)
    idx = recomendacionjuego[recomendacionjuego['id'] == game_id].index[0]
    rec_games = recomendacionjuego['app_name'].iloc[cosine_sim[idx]]
    info = {'recomendaciones': None}
    info['recomendaciones'] = list(rec_games)
    return info

In [5]:
recomendacion_juego(70)

{'recomendaciones': ['Zombie Panic',
  'Cannon Fodder 3',
  'Rogue Shooter: The FPS Roguelike Demo',
  'SiN Episodes: Emergence',
  'Half-Life 2: Lost Coast']}